## Solutions for ROS Control Project

<img src="../img/robotignite_logo_text.png"/>

## Index: 

* <a href="#SolutionStep2">Solution Step 2</a>
* <a href="#SolutionStep3">Solution Step 3</a>
* <a href="#SolutionStep4">Solution Step 4</a>

## Solution Step 2: Build the control package <p id="SolutionStep2"></p>

First of all, you'll need to create a new package.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
cd ~/catkin_ws/src;

In [ ]:
catkin_create_pkg project_control

Next, you will create 2 new folders inside your package: **config** and **launch**. Inside these folders, you will place the following files. Inside the **config** folder, you will place the **project_control.yaml** file, and inside the **launch** folder you will place the **project_control.launch** file.

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Param File: project_control.yaml** </p>

In [ ]:
ur5:

  joint_state_controller:
    type: joint_state_controller/JointStateController
    publish_rate: 50  

  shoulder_pan_joint_position_controller:
    type: position_controllers/JointPositionController
    joint: shoulder_pan_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}
    
  shoulder_lift_joint_position_controller:
    type: position_controllers/JointPositionController
    joint: shoulder_lift_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}
    
  elbow_joint_position_controller:
    type: position_controllers/JointPositionController
    joint: elbow_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}
  
  wrist_1_joint_position_controller:
    type: position_controllers/JointPositionController
    joint: wrist_1_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}
    
  wrist_2_joint_position_controller:
    type: position_controllers/JointPositionController
    joint: wrist_2_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}
    
  wrist_3_joint_position_controller:
    type: position_controllers/JointPositionController
    joint: wrist_3_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Param File: project_control.yaml** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: project_control.launch** </p>

In [ ]:
<launch>

  <!-- Load joint controller configurations from YAML file to parameter server -->
  <rosparam file="$(find project_control)/config/project_control.yaml" command="load"/>

  <!-- load the controllers -->
  <node name="controller_spawner" pkg="controller_manager" type="spawner" respawn="false"
	output="screen" ns="/ur5" args="joint_state_controller 
	                                shoulder_pan_joint_position_controller
					                        shoulder_lift_joint_position_controller
					                        elbow_joint_position_controller
					                        wrist_1_joint_position_controller
					                        wrist_2_joint_position_controller
					                        wrist_3_joint_position_controller
					                        "/>

</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: project_control.launch** </p>

## Solution Step 3: Test your package <p id="SolutionStep3"></p>

First of all, let's launch our ROS control package.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch project_control project_control.launch

If everything goes OK, you should get an output like this one:

<img src="../img/control_project1.png" width="800" />

And you will also get the following topics.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
rostopic list

<img src="../img/control_project2.png" width="600" />

Now, you will be able to control the joints of the robot by publishing into the corresponding topics. For instance, try to execute the following command

In [ ]:
rostopic pub /ur5/shoulder_lift_joint_position_controller/command std_msgs/Float64 "data: -1.5"

You will see the robot move to the following position:

<img src="../img/control_project3.png" width="500" />

## Solution Step 4: Create a controller <p id="SolutionStep4"></p>

### 1. Create the package

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

catkin_create_pkg project_controller roscpp pluginlib controller_interface hardware_interface

### 2. Create the source code

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Cpp File: project_controller.cpp** </p>

In [ ]:
#include <controller_interface/controller.h>
#include <hardware_interface/joint_command_interface.h>
#include <pluginlib/class_list_macros.h>

namespace controller_ns{

class PositionController : public controller_interface::Controller<hardware_interface::PositionJointInterface>
{
public:
  bool init(hardware_interface::PositionJointInterface* hw, ros::NodeHandle &n)
  {
    // get joint name from the parameter server
    std::string my_joint;
    if (!n.getParam("joint", my_joint)){
      ROS_ERROR("Could not find joint name");
      return false;
    }

    // get the joint object to use in the realtime loop
    joint_ = hw->getHandle(my_joint);  // throws on failure
    return true;
  }

  void update(const ros::Time& time, const ros::Duration& period)
  {
    double error = setpoint_ - joint_.getPosition();
    joint_.setCommand(pos_);
  }

  void starting(const ros::Time& time) { }
  void stopping(const ros::Time& time) { }

private:
  hardware_interface::JointHandle joint_;
  //static const double gain_ = 2.25;
  static const double setpoint_ = 1.00;
  static const double pos_ = -1.00;
};
PLUGINLIB_EXPORT_CLASS(controller_ns::PositionController, controller_interface::ControllerBase);
}//namespace

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Cpp File: project_controller.cpp** </p>

### 3. Create the plugin description file

<p style="background:#3B8F10;color:white;" id="prg-2-1">**XML File: controller_plugins.xml** </p>

In [ ]:
<library path="lib/libproject_controller_lib">
  <class name="project_controller/PositionController" 
         type="controller_ns::PositionController"           
         base_class_type="controller_interface::ControllerBase" />
</library>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END XML File: controller_plugins.xml** </p>

### 4. Update package.xml

<p style="background:#3B8F10;color:white;" id="prg-2-1">**XML File: package.xml** </p>

In [ ]:
<?xml version="1.0"?>
<package>
  <name>project_controller</name>
  <version>0.0.0</version>
  <description>The project_controller package</description>

  <!-- One maintainer tag required, multiple allowed, one person per tag --> 
  <!-- Example:  -->
  <!-- <maintainer email="jane.doe@example.com">Jane Doe</maintainer> -->
  <maintainer email="user@todo.todo">user</maintainer>


  <!-- One license tag required, multiple allowed, one license per tag -->
  <!-- Commonly used license strings: -->
  <!--   BSD, MIT, Boost Software License, GPLv2, GPLv3, LGPLv2.1, LGPLv3 -->
  <license>TODO</license>

  <buildtool_depend>catkin</buildtool_depend>
  <build_depend>controller_interface</build_depend>
  <build_depend>hardware_interface</build_depend>
  <build_depend>pluginlib</build_depend>
  <build_depend>roscpp</build_depend>
  <run_depend>controller_interface</run_depend>
  <run_depend>hardware_interface</run_depend>
  <run_depend>pluginlib</run_depend>
  <run_depend>roscpp</run_depend>


  <!-- The export tag contains other, unspecified, tags -->
  <export>
    <!-- Other tools can request additional information be placed here -->
    <controller_interface plugin="${prefix}/controller_plugins.xml"/>

  </export>
</package>


<p style="background:#3B8F10;color:white;" id="prg-2-1">**END XML File: package.xml** </p>

### 5. Update CMakeLists.txt

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Build section of CMakeLists.txt** </p>

In [ ]:
###########
## Build ##
###########

## Specify additional locations of header files
## Your package locations should be listed before other locations
# include_directories(include)
include_directories(
  ${catkin_INCLUDE_DIRS}
)

## Declare a C++ library
add_library(project_controller_lib src/project_controller.cpp)

## Add cmake target dependencies of the library
## as an example, code may need to be generated before libraries
## either from message generation or dynamic reconfigure
# add_dependencies(${PROJECT_NAME} ${${PROJECT_NAME}_EXPORTED_TARGETS} ${catkin_EXPORTED_TARGETS})

## Declare a C++ executable
## With catkin_make all packages are built within a single CMake context
## The recommended prefix ensures that target names across packages don't collide
# add_executable(${PROJECT_NAME}_node src/project_controller_node.cpp)

## Rename C++ executable without prefix
## The above recommended prefix causes long target names, the following renames the
## target back to the shorter version for ease of user use
## e.g. "rosrun someones_pkg node" instead of "rosrun someones_pkg someones_pkg_node"
# set_target_properties(${PROJECT_NAME}_node PROPERTIES OUTPUT_NAME node PREFIX "")

## Add cmake target dependencies of the executable
## same as for the library above
# add_dependencies(${PROJECT_NAME}_node ${${PROJECT_NAME}_EXPORTED_TARGETS} ${catkin_EXPORTED_TARGETS})

## Specify libraries to link a library or executable target against
target_link_libraries(project_controller_lib ${catkin_LIBRARIES})


<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Build section of CMakeLists.txt** </p>

### 6. Build the controller

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
roscd; cd ..;

In [ ]:
catkin_make

In [ ]:
source devel/setup.bash

If everything goes OK, you should be able to see your plugin when executing the following command:

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
rospack plugins --attrib=plugin controller_interface

<img src="../img/control_project_list.png" width="800" />

### 7. Modify the configuration file

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Param File: project_control.yaml** </p>

In [ ]:
ur5:

  joint_state_controller:
    type: joint_state_controller/JointStateController
    publish_rate: 50  

  shoulder_pan_joint_position_controller:
    type: position_controllers/JointPositionController
    joint: shoulder_pan_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}
    
  shoulder_lift_joint_position_controller:
    type: project_controller/PositionController
    joint: shoulder_lift_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}
    
  elbow_joint_position_controller:
    type: position_controllers/JointPositionController
    joint: elbow_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}
  
  wrist_1_joint_position_controller:
    type: position_controllers/JointPositionController
    joint: wrist_1_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}
    
  wrist_2_joint_position_controller:
    type: position_controllers/JointPositionController
    joint: wrist_2_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}
    
  wrist_3_joint_position_controller:
    type: position_controllers/JointPositionController
    joint: wrist_3_joint
    pid: {p: 100.0, i: 0.01, d: 10.0}

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Param File: project_control.yaml** </p>

### 8. Launch it

Finally, when your start your new control file:

<table style="float:left;background: #407EAF">
<tr>
<th>
Execute in WebShell #1
</th>
</tr>
</table>

In [ ]:
roslaunch project_control project_control.launch

You should see how the UR5 robot automatically moves to the following position:

<img src="../img/control_project3.png" width="500" />